In [124]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
# import hvplot.dask
import numpy as np
import dask.array as da
import pickle
import seaborn as sns

Additional features

- base rate booking -> # booked / # found
- best hotel for location
- aggregate competitor information
  - better or equal vs. worse
  - drop non-percentage difference
- expected position
- reduce children to binary flag?

- do factor analysis?


###### stuff that were important features for others

 09 : hotel's historical price - current price
data["ump"] = exp(prop_log_historical_price) - price_usd
12 :
data["score2ma"] = prop_location_score2 * srch_query_affinity_score
 13 :
data["score1d2"] = (prop_location_score2 + 0.0001)/(prop_location_score1 + 0.0001)

data["fee_per_person"] = (price_usd * srch_room_count)/(srch_adults_count + srch_children_count)

- categorical to one-hot encoding

In [125]:
df = pd.read_csv("data/training_set_VU_DM.csv")

In [126]:
# 1. Base rate booking column
grouped_hotels = df.groupby("prop_id").sum(numeric_only=True)
temp = pd.DataFrame()
temp["prob_clickbook"] = grouped_hotels["booking_bool"]/ grouped_hotels["click_bool"]
temp.fillna(0,inplace=True)
temp =temp.reset_index()
df = df.merge(temp, on="prop_id")
del temp

In [127]:
# 2. Percentile rank of hotel in its country
grouped_hotels = df.groupby(["prop_id","prop_country_id"]).mean(numeric_only=True).reset_index()
temp = grouped_hotels[["prop_starrating", "prop_review_score","prop_id","prop_country_id"]]
temp["total_score"] = temp.loc[:,"prop_starrating"] + temp.loc[:,"prop_review_score"]
temp["country_pct_rank"] = temp.groupby("prop_country_id")["total_score"].rank(method="dense", ascending=True,pct=True)
temp = temp[["prop_id","country_pct_rank"]]
df = df.merge(temp, on="prop_id")
del temp

/var/folders/z9/j9843dsj2t5dsl2bv81xcwww0000gn/T/ipykernel_30801/3161763591.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["total_score"] = temp.loc[:,"prop_starrating"] + temp.loc[:,"prop_review_score"]
/var/folders/z9/j9843dsj2t5dsl2bv81xcwww0000gn/T/ipykernel_30801/3161763591.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["country_pct_rank"] = temp.groupby("prop_country_id")["total_score"].rank(method="dense", ascending=True,pct=True)


In [128]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [129]:
# 3.1 Aggregate competitors: Is there a same or better competitor?
# 3.2 Competitor pressure: average competitor score

# a. predict rate_cols using rate_pec_columns
inv_cols = [f"comp{i}_inv" for i in np.arange(1,9)]

rate_cols = [f"comp{i}_rate" for i in np.arange(1,9)]
rate_pec_cols = [f"comp{i}_rate_percent_diff" for i in np.arange(1,9)]
temp = df[rate_cols + rate_pec_cols]
# temp = df[flatten([rate_cols,inv_cols,rate_pec_cols])]
temp
def interpolate_rate(data_frame, col):
    data_frame.set_index('wind_speed', inplace=True)

    data_frame.interpolate(method='index', inplace=True)

    data_frame.reset_index(inplace=True)

# b.

In [130]:
temp

,comp1_rate,comp2_rate,comp3_rate,comp4_rate,comp5_rate,comp6_rate,comp7_rate,comp8_rate,comp1_inv,comp2_inv,...,comp7_inv,comp8_inv,comp1_rate_percent_diff,comp2_rate_percent_diff,comp3_rate_percent_diff,comp4_rate_percent_diff,comp5_rate_percent_diff,comp6_rate_percent_diff,comp7_rate_percent_diff,comp8_rate_percent_diff
0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.0,0.0,NaN,NaN,NaN,NaN,-1.0,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4958343,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4958344,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4958345,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN


In [131]:
# 4. Searchbook


In [132]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,prob_clickbook,country_pct_rank
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.65
1,498,2012-12-23 11:32:22,32,220,NaN,NaN,219,893,3,3.5,...,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.65
2,896,2013-05-09 22:14:57,5,2,NaN,NaN,219,893,3,3.5,...,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.65
3,1495,2013-04-24 02:24:02,5,219,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.625,0.65
4,2052,2013-01-11 08:10:27,5,219,NaN,NaN,219,893,3,3.5,...,NaN,NaN,-1.0,0.0,6.0,0,NaN,0,0.625,0.65


In [133]:
temp[temp["prop_country_id"]==1][["rank","total_score"]]

KeyError: 'prop_country_id'

In [ ]:
temp.groupby("prop_country_id").rank()

In [ ]:
grouped_countries = temp.groupby("prop_country_id")a

In [ ]:
grouped_countries = temp.groupby("prop_country_id")
temp["rank"] = grouped_countries["total_score"].rank(ascending=False)

# Step 3: Sort the DataFrame by "prop_id" and "prop_country_id" and reset the index
temp = temp.sort_values(by=["total_score"]).reset_index()
temp

In [ ]:
temp[temp["prop_country_id"]==219]